# 2025 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [ ]:
#############Colab Parameters###########
TFIDF_max_features = 5000

In [83]:
import os
import json

data_file = "/content/data"
data_file = "data"
train_file = os.path.join(data_file, 'train-claims.json')
dev_file = os.path.join(data_file, 'dev-claims.json')
evidence_file = os.path.join(data_file, 'evidence.json')
test_file = os.path.join(data_file, 'test-claims-unlabelled.json')

with open(train_file, 'r') as f:
    tr_claims = json.load(f)
tr_ids = list(tr_claims.keys())
tr_texts = [tr_claims[claim_id]['claim_text'] for claim_id in tr_ids]

with open(dev_file, 'r') as f:
    dev_claims = json.load(f)
dev_ids = list(dev_claims.keys())
dev_texts = [dev_claims[claim_id]['claim_text'] for claim_id in dev_ids]

with open(evidence_file, 'r') as f:
    evidences = json.load(f)
evidences_ids = list(evidences.keys())
evidences_texts = [evidences[evidence_id] for evidence_id in evidences_ids]

with open(test_file, 'r') as f:
    test_claims = json.load(f)
ts_ids = list(test_claims.keys())
ts_texts = [test_claims[claim_id]['claim_text'] for claim_id in ts_ids]

print("Train claims:", len(tr_claims))
print("Dev claims:", len(dev_claims))   
print("Evidences:", len(evidences))
print("Test claims:", len(test_claims))

Train claims: 1228
Dev claims: 154
Evidences: 1208827
Test claims: 153


In [32]:
# ####Local test#####
# import os
# import json

# train_file = "data/train-claims.json"
# with open(train_file, 'r') as f:
#     tr_claims = json.load(f)
    

# print("Train claims:", len(tr_claims))
# id = list(tr_claims.keys())[0]
# print("Train claim id:", id)
# tr_claim_test = tr_claims[id]
# tr_claim_test

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [33]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')

lemmatizer = WordNetLemmatizer()
def word_tokenize_and_lemmatize(text):
    def lemmatize_word(token):
        token = lemmatizer.lemmatize(token, pos='v')
        token = lemmatizer.lemmatize(token, pos='n') if token != token else token
        return token
    
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    lemmed_tokens = [lemmatize_word(token) for token in tokens]
    return lemmed_tokens

[nltk_data] Downloading package stopwords to C:\Users\Salist
[nltk_data]     desk2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Salist
[nltk_data]     desk2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [34]:
# ####Local test#####
# word_tokenize_and_lemmatize(tr_claim_test['claim_text'])

In [35]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from scipy import sparse
# import joblib

# evidence_ids = evidences.keys()
# evidence_text_list = [evidences[eid] for eid in evidence_ids]
# evidence_vectorizer = TfidfVectorizer(tokenizer=word_tokenize_and_lemmatize, max_features=5000)
# evidence_vectors = evidence_vectorizer.fit_transform(evidence_text_list)

# sparse.save_npz("evidence_vectors.npz", evidence_vectors)
# joblib.dump(evidence_vectorizer, "evidence_vectorizer.pkl")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import joblib

def save_tfidf_model(text_list, path_name):
    vectorizer = TfidfVectorizer(tokenizer=word_tokenize_and_lemmatize, max_features=5000)
    text_ids = list(text_list.keys())
    text_list = [text_list[id] for id in text_ids]

    tfidf_matrix = vectorizer.fit_transform(text_list)
    vector_path=str(path_name + "_vectors.npz")
    model_path=str(path_name + "_vectorizer.pkl")
    sparse.save_npz(vector_path, tfidf_matrix)
    joblib.dump(vectorizer, model_path)
    print(f"Saved TF-IDF vectors to '{vector_path}' and vectorizer to '{model_path}'")

In [ ]:
#######RUN ON COLAB##########
# save_tfidf_model(evidences, "evidences")

In [ ]:
from scipy import sparse
import joblib

def load_tfidf(path_name):
    vector_path=str(path_name + "_vectors.npz")
    model_path=str(path_name + "_vectorizer.pkl")
    tfidf_matrix = sparse.load_npz(vector_path)
    vectorizer = joblib.load(model_path)
    print(f"Loaded TF-IDF matrix from '{vector_path}' and vectorizer from '{model_path}'")
    return tfidf_matrix, vectorizer

In [ ]:
# from scipy import sparse
# import joblib
# # sparse.save_npz("evidence_vectors.npz", evidence_vectors)
# evidence_vectors = sparse.load_npz("evidence_vectors.npz")
# evidence_vectorizer = joblib.load("evidence_vectorizer.pkl")

In [ ]:
######build vocab for train and evidence######
from collections import Counter
from nltk.tokenize import word_tokenize
from tqdm import tqdm 

def yield_tokens(data_iter):
    for text in data_iter:
        yield word_tokenize(text.lower())

# Build vocabulary manually using Counter
def build_vocab_from_iterator(token_iterator, min_freq, specials_tokens=('<cls>', '<unk>', '<pad>')):
    counter = Counter()
    for tokens in token_iterator:
        counter.update(tokens)

    vocab = {special_token: idx for idx, special_token in enumerate(specials_tokens)}
    current_index = len(specials_tokens)
    for token, freq in counter.items():
        if freq >= min_freq: # only include tokens with frequency >= min_freq
            vocab[token] = current_index
            current_index += 1

    stoi = vocab
    itos = {idx: token for token, idx in stoi.items()}
    return vocab, stoi, itos

# Create the vocab for train and evidence
vocab, string_to_index, index_to_string = build_vocab_from_iterator(yield_tokens(tr_texts + evidences_texts), min_freq=5)
print("Vocab size:", len(vocab))

def text_to_indices(text, vocab):
    tokens = word_tokenize(text.lower())
    indices = [vocab.get(token, vocab["<unk>"]) for token in tokens]
    return [vocab["<cls>"]] + indices

tr_text_indices = [text_to_indices(text, vocab) for text in tr_texts]
dev_text_indices = [text_to_indices(text, vocab) for text in dev_texts]
ts_text_indices = [text_to_indices(text, vocab) for text in ts_texts]

evidences_text_indices = [text_to_indices(text, vocab) for text in tqdm(evidences_texts)]

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [59]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

evidences_vectors, evidences_vectorizer = load_tfidf("evidences")
dev_ids = list(dev_claims.keys())
dev_claims_vectors = evidences_vectorizer.transform([dev_claims[id]['claim_text'] for id in dev_ids])

def top_k_evidence(vectors, k=5):
    similarity = cosine_similarity(vectors, evidences_vectors)
    top_k_indices = np.argsort(similarity, axis=1)[:, -k:]
    top_k_indices = np.flip(top_k_indices, axis=1)
    return top_k_indices

dev_top5_evidence_id = top_k_evidence(dev_claims_vectors, k=5)

Loaded TF-IDF matrix from 'evidences_vectors.npz' and vectorizer from 'evidences_vectorizer.pkl'


In [80]:
def evaluate_retrival(dev_claims, top_evidence_id):
    evidence_f1 = []
    for i, claim_id in enumerate(dev_claims.keys()):
        correct = 0
        recall = 0.0
        precision = 0.0
        fscore = 0.0
        
        claim = dev_claims[claim_id]
        true_evidence_ids = claim['evidences']
        for true_evidence in true_evidence_ids:
            true_evidence_id = int(true_evidence.split('-')[1])
            if true_evidence_id in top_evidence_id:
                correct += 1
        if correct > 0:
            recall = correct / len(true_evidence_ids)
            precision = correct / len(top_evidence_id[i])
            fscore = (2 * precision * recall) / (precision + recall)
        evidence_f1.append(fscore)
    return evidence_f1

dev_top5_evidence_f1 = evaluate_retrival(dev_claims, dev_top5_evidence_id)
print("Dev claims F1 score:", np.mean(dev_top5_evidence_f1))

Dev claims F1 score: 0.08017419088847662
